In [25]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as ss
import pandas as pd

# NH data import tools
from ppg2rr.rr_est import import_ppg
from ppg2rr.config import AlgorithmParams

# my stuff
from probrr.extract import Extract
from probrr.sequence import Sequence

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# The number of frames are printed out to help pinpoint the location of failure.
dataset='capnobase'
trial = 15   # For CapnoBase, this is the index of the files sorted by filename; the participant ID is the first part of each filename
# mh notes:
# 5: easy one, but berger is good too
# 7: bad PPG peak picking :(
# 9: small- and large-timescale variations
# 20: good test for false variations!  also tricky negative-peak detection on the PPG
# 24: NH algorithm had difficulty.  would this help, vs. berger?  probably not

params = AlgorithmParams(
    dataset=dataset,
    probe=1,
    remove_riv_outliers="segment-wise",
    peak_counting_prominence=0.5,
    lowpass_cutoff_ppg=35
    # window_increment=15
)

ppg, Fs_ppg, ref_dict, meta = import_ppg.load_ppg_data(
    dataset=dataset,
    params=params,
    trial=trial,
    window_size=params.window_size,
    probe_num=params.probe,
    read_cpo=params.cpo,
    show=True
)

# Dataset-provided capnography and breath (looks like inhalation) picks
Fs_co2 = ref_dict['co2']['fs']
co2 = ref_dict['co2']['y_raw']
t_co2 = np.arange(len(co2)) / Fs_co2
#t_breathmark = ref_dict['rr']['x_raw']
#y_breathmark = ref_dict['rr']['y_raw']

# Resample to a reasonable rate
decFactor = 10
Fs_ppg = Fs_ppg / decFactor
ppg = ss.decimate(ppg, decFactor)
co2 = ss.decimate(co2, decFactor)
t_ppg = np.arange(len(ppg)) / Fs_ppg

# Extract RI*V signals
fc = 0.49    # digital cutoff frequency for the iterative reconstruction from irregular samples
ext = Extract(t_ppg, ppg, co2, fc=fc)
T_hr = 1/ext.get_hr()
(riiv, ppg_hp) = ext.get_riiv()
(riav, ppgMinTimes, ppgMinVals, ppgMaxTimes, ppgMaxVals, riavTimes, upDeltas) = ext.get_riav()
(rifv, rifvTimes) = ext.get_rifv()

# Pick breaths on capnography
(T_rr, co2MinTimes, co2MinVals) = ext.pick_capno()

# Sequence-based peak picking
riiv_seq = Sequence(t_ppg, riiv)
riav_seq = Sequence(t_ppg, riav)
rifv_seq = Sequence(t_ppg, rifv)

In [28]:
firstRun = True     # this will be cleared by the next cell so that the window slides with each run

In [ ]:
maxInterBreathInterval = 8      # No peak will be chosen if it comes later than this after the last peak

print(meta)
print('RR = {:.2f} Hz [{:.1f} rpm]'.format(1/T_rr, 60/T_rr))
print('HR = {:.2f} Hz [{:.1f} bpm] = {:.2f}*RR'.format(1/T_hr, 60/T_hr, T_rr/T_hr))
print('RIAV/RIFV reconstruction Fc = {:.2f}/T_hr = {:.2f} Hz ({:.2f} s)'.format(fc, fc/T_hr, T_hr/fc))

# if firstRun:
#     firstRun = False
#     startTime = 0
# else:
#     startTime += 10
startTime = 0

stopTime = startTime + 30
startInd = int(startTime*Fs_ppg)
stopInd = int(startInd + (stopTime-startTime)*Fs_ppg)
N = stopInd - startInd

f, axs = plt.subplots(3, 1, figsize=(20,8))   # create stacked plots

axs[0].plot(t_ppg[startInd:stopInd], riiv[startInd:stopInd], zorder=2.5, label='baseline')
axs[0].plot(t_ppg[startInd:stopInd], ppg[startInd:stopInd], label='ppg')
for x in ppgMaxTimes:
    axs[0].axvline(x=x, color='#ffd0d0')

axs[1].plot(t_ppg[startInd:stopInd], ppg_hp[startInd:stopInd], label='HPppg')
axs[1].plot(ppgMinTimes, ppgMinVals, 'x', color='#a0a000')
axs[1].plot(ppgMaxTimes, ppgMaxVals, 'x', color='#a000a0')
axs[1].plot(t_ppg[startInd:stopInd], riav[startInd:stopInd], label='ampl')
axs[1].plot(rifvTimes, 5/np.diff(ppgMaxTimes), 'x', color='#00a0a0')
axs[1].plot(t_ppg[startInd:stopInd], 5*rifv[startInd:stopInd], label='freqX5')
axs[1].plot(riavTimes, upDeltas, 'x')

axs[2].plot(t_ppg[startInd:stopInd], co2[startInd:stopInd], label='CO2')
axs[2].plot(co2MinTimes, co2MinVals, 'kx')

for ax in axs:
    # for x in co2MinTimes:
    #     ax.axvline(x=x, color='#ffb0b0')
    ax.grid(color='#dddddd')
    ax.legend(loc='lower left')
    ax.set_xlim(t_ppg[startInd], t_ppg[stopInd])
axs[2].legend(loc='center left')

plt.show()

In [ ]:

if len(riiv_seq.picked) == 0 or startTime == 0:
    lastPick = None
    print('\nRIIV: fresh start')
else:
    lastPick = riiv_seq.picked[ np.searchsorted(riiv_seq.peakTimes[riiv_seq.picked], startTime) - 1 ]
    print('\nRIIV: last pick at {:.2f}s'.format(riiv_seq.peakTimes[lastPick]))
riiv_seq.enumerate( ((), 0.0), lastPick, stopTime, maxInterBreathInterval, 0.6 )
print('Evaluated {} sequences; winner is {}'.format(len(riiv_seq.seqSet), riiv_seq.leader))

if len(riav_seq.picked) == 0 or startTime == 0:
    lastPick = None
    print('\nRIAV: fresh start')
else:
    lastPick = riav_seq.picked[ np.searchsorted(riav_seq.peakTimes[riav_seq.picked], startTime) - 1 ]
    print('\nRIAV: last pick at {:.2f}s'.format(riav_seq.peakTimes[lastPick]))
riav_seq.enumerate( ((), 0.0), lastPick, stopTime, maxInterBreathInterval, 0.6 )
print('Evaluated {} sequences; winner is {}'.format(len(riav_seq.seqSet), riav_seq.leader))

if len(rifv_seq.picked) == 0 or startTime == 0:
    lastPick = None
    print('\nRIFV: fresh start')
else:
    lastPick = rifv_seq.picked[ np.searchsorted(rifv_seq.peakTimes[rifv_seq.picked], startTime) - 1 ]
    print('\nRIFV: last pick at {:.2f}s'.format(rifv_seq.peakTimes[lastPick]))
rifv_seq.enumerate( ((), 0.0), lastPick, stopTime, maxInterBreathInterval, 0.6 )
print('Evaluated {} sequences; winner is {}'.format(len(rifv_seq.seqSet), rifv_seq.leader))

f, axs = plt.subplots(6, 1, figsize=(20,20))   # create stacked plots

# mark reference breaths
#tmp = np.searchsorted(t_breathmark, (t_ppg[startInd], t_ppg[stopInd])) 
#for x in t_breathmark[tmp[0]:tmp[1]]:

axs[0].plot(t_ppg[startInd:stopInd], riiv[startInd:stopInd], zorder=2.5, label='baseline')
axs[0].plot(t_ppg[startInd:stopInd], ppg[startInd:stopInd], label='ppg')
for x in ppgMaxTimes:
    axs[0].axvline(x=x, color='#ffd0d0')

axs[1].plot(t_ppg[startInd:stopInd], ppg_hp[startInd:stopInd], label='HPppg')
axs[1].plot(ppgMinTimes, ppgMinVals, 'x', color='#a0a000')
axs[1].plot(ppgMaxTimes, ppgMaxVals, 'x', color='#a000a0')
axs[1].plot(t_ppg[startInd:stopInd], riav[startInd:stopInd], label='ampl')
axs[1].plot(rifvTimes, 5/np.diff(ppgMaxTimes), 'x', color='#00a0a0')
axs[1].plot(t_ppg[startInd:stopInd], 5*rifv[startInd:stopInd], label='freqX5')
axs[1].plot(riavTimes, upDeltas, 'x')

axs[2].plot(t_ppg[startInd:stopInd], riiv[startInd:stopInd], label='riiv')
axs[2].plot(riiv_seq.peakTimes[  riiv_seq.firstMax::2], riiv[riiv_seq.peakInds[  riiv_seq.firstMax::2]], 'x')
axs[2].plot(riiv_seq.peakTimes[1+riiv_seq.firstMax::2], riiv[riiv_seq.peakInds[1+riiv_seq.firstMax::2]], 'x')
axs[2].plot(riiv_seq.peakTimes[riiv_seq.picked], riiv[riiv_seq.peakInds[riiv_seq.picked]], 'ko', fillstyle='none', markersize=12)
#axs[2].plot(riiv_seq.peakTimes, 5*riiv_seq.peakProms, 'x')

axs[3].plot(t_ppg[startInd:stopInd], riav[startInd:stopInd], label='riav')
axs[3].plot(riav_seq.peakTimes[  riav_seq.firstMax::2], riav[riav_seq.peakInds[  riav_seq.firstMax::2]], 'x')
axs[3].plot(riav_seq.peakTimes[1+riav_seq.firstMax::2], riav[riav_seq.peakInds[1+riav_seq.firstMax::2]], 'x')
axs[3].plot(riav_seq.peakTimes[riav_seq.picked], riav[riav_seq.peakInds[riav_seq.picked]], 'ko', fillstyle='none', markersize=12)

axs[4].plot(t_ppg[startInd:stopInd], rifv[startInd:stopInd], label='rifv')
axs[4].plot(rifv_seq.peakTimes[  rifv_seq.firstMax::2], rifv[rifv_seq.peakInds[  rifv_seq.firstMax::2]], 'x')
axs[4].plot(rifv_seq.peakTimes[1+rifv_seq.firstMax::2], rifv[rifv_seq.peakInds[1+rifv_seq.firstMax::2]], 'x')
axs[4].plot(rifv_seq.peakTimes[rifv_seq.picked], rifv[rifv_seq.peakInds[rifv_seq.picked]], 'ko', fillstyle='none', markersize=12)

axs[5].plot(t_ppg[startInd:stopInd], co2[startInd:stopInd], label='CO2')
axs[5].plot(co2MinTimes, co2MinVals, 'kx')

for ax in axs:
    for x in co2MinTimes:
        ax.axvline(x=x, color='#ffb0b0')
    ax.grid(color='#dddddd')
    ax.legend(loc='lower left')
    ax.set_xlim(t_ppg[startInd], t_ppg[stopInd])
axs[5].legend(loc='center left')

plt.show()

In [ ]:
ts(6, 1, figsize=(20,20))   # create stacked plots

# mark reference breaths
#tmp = np.searchsorted(t_breathmark, (t_ppg[startInd], t_ppg[stopInd])) 
#for x in t_breathmark[tmp[0]:tmp[1]]:

axs[0].plot(t_ppg[startInd:stopInd], riiv[startInd:stopInd], zorder=2.5, label='baseline')
axs[0].plot(t_ppg[startInd:stopInd], ppg[startInd:stopInd], label='ppg')
for x in ppgMaxTimes:
    axs[0].axvline(x=x, color='#ffb0b0')

axs[1].plot(t_ppg[startInd:stopInd], ppg_hp[startInd:stopInd], label='HPppg')
axs[1].plot(ppgMinTimes, ppgMinVals, 'x', color='#a0a000')
axs[1].plot(ppgMaxTimes, ppgMaxVals, 'x', color='#a000a0')
axs[1].plot(t_ppg[startInd:stopInd], riav[startInd:stopInd], label='ampl')
axs[1].plot(rifvTimes, 5/np.diff(ppgMaxTimes), 'x', color='#00a0a0')
axs[1].plot(t_ppg[startInd:stopInd], 5*rifv[startInd:stopInd], label='freqX5')
axs[1].plot(riavTimes, upDeltas, 'x')

axs[2].plot(t_ppg[startInd:stopInd], riiv[startInd:stopInd], label='riiv')
axs[2].plot(riiv_seq.peakTimes[  riiv_seq.firstMax::2], riiv[riiv_seq.peakInds[  riiv_seq.firstMax::2]], 'x')
axs[2].plot(riiv_seq.peakTimes[1+riiv_seq.firstMax::2], riiv[riiv_seq.peakInds[1+riiv_seq.firstMax::2]], 'x')
axs[2].plot(riiv_seq.peakTimes[riiv_seq.picked], riiv[riiv_seq.peakInds[riiv_seq.picked]], 'ko', fillstyle='none', markersize=12)
#axs[2].plot(riiv_seq.peakTimes, 5*riiv_seq.peakProms, 'x')

axs[3].plot(t_ppg[startInd:stopInd], riav[startInd:stopInd], label='riav')
axs[3].plot(riav_seq.peakTimes[  riav_seq.firstMax::2], riav[riav_seq.peakInds[  riav_seq.firstMax::2]], 'x')
axs[3].plot(riav_seq.peakTimes[1+riav_seq.firstMax::2], riav[riav_seq.peakInds[1+riav_seq.firstMax::2]], 'x')
axs[3].plot(riav_seq.peakTimes[riav_seq.picked], riav[riav_seq.peakInds[riav_seq.picked]], 'ko', fillstyle='none', markersize=12)

axs[4].plot(t_ppg[startInd:stopInd], rifv[startInd:stopInd], label='rifv')
axs[4].plot(rifv_seq.peakTimes[  rifv_seq.firstMax::2], rifv[rifv_seq.peakInds[  rifv_seq.firstMax::2]], 'x')
axs[4].plot(rifv_seq.peakTimes[1+rifv_seq.firstMax::2], rifv[rifv_seq.peakInds[1+rifv_seq.firstMax::2]], 'x')
axs[4].plot(rifv_seq.peakTimes[rifv_seq.picked], rifv[rifv_seq.peakInds[rifv_seq.picked]], 'ko', fillstyle='none', markersize=12)

axs[5].plot(t_ppg[startInd:stopInd], co2[startInd:stopInd], label='CO2')
axs[5].plot(co2MinTimes, co2MinVals, 'kx')

for ax in axs:
    for x in co2MinTimes:
        ax.axvline(x=x, color='#ffb0b0')
    ax.grid(color='#dddddd')
    ax.legend(loc='lower left')
    ax.set_xlim(t_ppg[startInd], t_ppg[stopInd])
axs[5].legend(loc='center left')

plt.show()

In [ ]:
print(riiv_seq.picked)
print(riav_seq.picked)
print(rifv_seq.picked)

seq = rifv_seq
if len(seq.picked) == 0:
    print("Empty")
else:
    print((len(seq.picked)-1) / (seq.peakTimes[seq.picked[-1]] - seq.peakTimes[seq.picked[0]]))

    numBreaths = np.searchsorted(co2MinTimes, seq.peakTimes[seq.picked[-1]])
    print(numBreaths / (seq.peakTimes[seq.picked[-1]] - seq.peakTimes[seq.picked[0]]))

In [ ]:
seq = riiv_seq
seq.test(seq.picked, None)

s = [ np.exp(a[1]/len(a[0])) for a in seq.seqSet ]
plt.hist(s, bins=100)
plt.show()

In [ ]:
#tm = 1.5
tstar = 2.41
tp = 4.81
delta = 0

#delta = (tstar*tstar - tm*tp) / (2*tstar - tm - tp)
#sigma2 = 0.18033688 * np.square(np.log((tp - delta) / (tm - delta)))
#tstar = delta + (tp - delta)*np.exp(-1.1774100*np.sqrt(sigma2))
sigma2 = 0.72134752 * np.square(np.log((tp - delta) / (tstar - delta)))
mu = sigma2 + np.log(tstar - delta)
print('mu = {:.2f}   sigma2 = {:.2f}   tstar = {:.2f}   delta = {:.2f}'.format(
    mu, sigma2, tstar, delta))

t = np.arange(0, 500, .01)
f = np.zeros(t.size)
f[t>delta] = np.exp(mu - 0.5*sigma2) * np.exp(-0.5*np.square(np.log(t[t>delta] - delta) - mu)/sigma2) / (t[t>delta] - delta)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10,10))   # create stacked plots

ax1.plot(t, f)
ax1.grid()
ax1.set_xlabel('Inter-breath interval (s)')
ax1.set_xlim(0, 30)
ax1.set_ylim(0, 1)

ax2.plot(60/(t + 1e-20), f)
ax2.grid()
ax2.set_xlabel('Respirations per minute')
ax2.set_xlim(10, 80)
ax2.set_ylim(0, 1)
plt.show()

In [ ]:
from scipy.special import erf

x = np.linspace(-1,4,1000)
plt.plot(x, 0.5 + 0.5*erf(1.0*(x-0.7)))
plt.grid()
plt.show()
